In [1062]:
# import packages
import requests
import pandas as pd
import numpy as np
# !pip install folium
import folium

from pandas import json_normalize 
import datetime as dt
from datetime import date
import pytz
from dateutil.parser import parse
# !pip install maya
import maya

# from contextlib import suppress
from scipy import stats

In [924]:
class DatetimeRange:
    def __init__(self, dt1, dt2):
        self._dt1 = dt1
        self._dt2 = dt2

    def __contains__(self, dt):
        return self._dt1 <= dt <= self._dt2

In [925]:

# import urllib.request, json 
# with urllib.request.urlopen("https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ca_california_zip_codes_geo.min.json") as url:
#     data = json.loads(url.read().decode())
#     print(data)

pd.set_option('display.max_colwidth', None)

In [926]:

url = "https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ca_california_zip_codes_geo.min.json"
df = pd.read_json(url)
df.head()

type  \
0  FeatureCollection   
1  FeatureCollection   
2  FeatureCollection   
3  FeatureCollection   
4  FeatureCollection   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [927]:
# with open('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ca_california_zip_codes_geo.min.json', 'r') as jsonFile:
#     data = json.load(jsonFile)
# tmp = data

In [928]:
# removes ZIP codes not in our dataset 
geozips = []
for i in range(len(df['features'])):
    if df['features'][i]['properties']['ZCTA5CE10'] in ['95054', '95110', '94085', '95051', '94086', '94087']: # type(list(merged['zipcode'].unique()))
        geozips.append(df['features'][i])

# creating new JSON object 
new_json = dict.fromkeys(['type'], 'features')
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

# save JSON object as updated-file 
open ("updated-file.json", "w").write(
    json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': '))
)

495941

In [929]:
range(len(df['features']))

range(0, 1769)

In [930]:
['95054']

['95054']

In [931]:
geozips

[{'type': 'Feature',
  'properties': {'STATEFP10': '06',
   'ZCTA5CE10': '94085',
   'GEOID10': '0694085',
   'CLASSFP10': 'B5',
   'MTFCC10': 'G6350',
   'FUNCSTAT10': 'S',
   'ALAND10': 9039193,
   'AWATER10': 0,
   'INTPTLAT10': '+37.3885556',
   'INTPTLON10': '-122.0177396',
   'PARTFLG10': 'N'},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-122.01885, 37.38049],
     [-122.019047, 37.380621],
     [-122.01984, 37.38115],
     [-122.019946, 37.381217],
     [-122.02025, 37.38141],
     [-122.02057, 37.381562],
     [-122.02067, 37.38161],
     [-122.021098, 37.381789],
     [-122.021564, 37.381938],
     [-122.02157, 37.38194],
     [-122.02191, 37.38202],
     [-122.0221, 37.382059],
     [-122.022669, 37.382179],
     [-122.02286, 37.38222],
     [-122.02334, 37.38232],
     [-122.02588, 37.38287],
     [-122.0266, 37.38302],
     [-122.02668, 37.38304],
     [-122.02689, 37.38308],
     [-122.027786, 37.383265],
     [-122.028168, 37.383342999999996],
     [-122.02893,

In [932]:
new_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'STATEFP10': '06',
    'ZCTA5CE10': '94085',
    'GEOID10': '0694085',
    'CLASSFP10': 'B5',
    'MTFCC10': 'G6350',
    'FUNCSTAT10': 'S',
    'ALAND10': 9039193,
    'AWATER10': 0,
    'INTPTLAT10': '+37.3885556',
    'INTPTLON10': '-122.0177396',
    'PARTFLG10': 'N'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-122.01885, 37.38049],
      [-122.019047, 37.380621],
      [-122.01984, 37.38115],
      [-122.019946, 37.381217],
      [-122.02025, 37.38141],
      [-122.02057, 37.381562],
      [-122.02067, 37.38161],
      [-122.021098, 37.381789],
      [-122.021564, 37.381938],
      [-122.02157, 37.38194],
      [-122.02191, 37.38202],
      [-122.0221, 37.382059],
      [-122.022669, 37.382179],
      [-122.02286, 37.38222],
      [-122.02334, 37.38232],
      [-122.02588, 37.38287],
      [-122.0266, 37.38302],
      [-122.02668, 37.38304],
      [-122.02689, 37.38308],
      [-122.02

In [933]:

# table = main table/data frame we read from (pandas DataFrame)
# zips = column name where ZIP codes are (string)
# mapped_feature = column name for feature we want to visualize (string)
# add_text = any additional commentary to be added in the map legend (string)

def create_map(table, zips, mapped_feature, add_text = '', delimiter = ''):
    # reading of the updated GeoJSON file
    sf_geo = r'updated-file.json'
    # initiating a Folium map with 95054's latitude and longitude
    m = folium.Map(location = [37.393, -121.961], zoom_start = 11) 
    # creating a choropleth map
    folium.Choropleth(
        geo_data = sf_geo,
        fill_opacity = 0.54,
        line_opacity = 0.38,
        data = table,
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.ZCTA5CE10',
        # first element contains location information, second element contains feature of interest
        columns = [zips, mapped_feature],
        fill_color = 'PiYG',
        threshold_scale = [-45, -30, -15, 0, 15, 30, 45],
        legend_name = (' ').join(mapped_feature.split('_')).title() + ' ' + add_text + ' across SF Bay Area by ' + delimiter
    ).add_to(m)
#     folium.LayerControl().add_to(m)
    # save map with filename based on the feature of interest
    display(m)
    m.save(outfile = mapped_feature + '_map.html')

In [934]:
def create_map2(table, zips, mapped_feature, add_text = '', delimiter = ''):
    # reading of the updated GeoJSON file
    sf_geo = r'updated-file.json'
    # initiating a Folium map with 95054's latitude and longitude
    m = folium.Map(location = [37.393, -121.961], zoom_start = 11) 
    # creating a choropleth map
    folium.Choropleth(
        geo_data = sf_geo,
        fill_opacity = 0.54,
        line_opacity = 0.38,
        data = table,
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.ZCTA5CE10',
        # first element contains location information, second element contains feature of interest
        columns = [zips, mapped_feature],
        fill_color = 'RdYlGn',
        threshold_scale = [-40, -30, -20, 0, 10, 20, 30],
        legend_name = (' ').join(mapped_feature.split('_')).title() + ' ' + add_text + ' across SF Bay Area by ' + delimiter
    ).add_to(m)
#     folium.LayerControl().add_to(m)
    # save map with filename based on the feature of interest
    display(m)
    m.save(outfile = mapped_feature + '_map.html')

In [935]:
#95054 
url = "https://realty-mole-property-api.p.rapidapi.com/salePrice"

querystring = {"compCount":"10","squareFootage":"870","bathrooms":"1","address":"1883 Agnew Rd Unit 318, Santa Clara, CA","bedrooms":"1","propertyType":"Condo"}

headers = {
    'x-rapidapi-key': "ca0f4a0c63msheb56ce445093e55p1d9232jsn4658d02eedac",
    'x-rapidapi-host': "realty-mole-property-api.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

# print(response.text)
print(response.json())

{'price': 548932.8, 'priceRangeLow': 508107.94, 'priceRangeHigh': 589757.66, 'longitude': -121.9582871, 'latitude': 37.3958195, 'listings': [{'id': '1883-Agnew-Rd,-Unit-247,-Santa-Clara,-CA-95054', 'formattedAddress': '1883 Agnew Rd, Unit 247, Santa Clara, CA 95054', 'longitude': -121.958622, 'latitude': 37.395726, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 598888, 'publishedDate': '2020-12-28T11:26:59.931Z', 'distance': 0.03139360151479152, 'daysOld': 140.83, 'correlation': 0.9989, 'address': '1883 Agnew Rd, Unit 247', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 870}, {'id': '2200-Agnew-Rd,-Apt-309,-Santa-Clara,-CA-95054', 'formattedAddress': '2200 Agnew Rd, Apt 309, Santa Clara, CA 95054', 'longitude': -121.96357, 'latitude': 37.392453, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 499000, 'publishedDate': '2021-05-16T13:10:11.218Z', 'distance': 0.5989462621367421, 'daysOld': 1.7

In [936]:
sale_estimate_pdf = json_normalize(response.json())
print (sale_estimate_pdf)


      price  priceRangeLow  priceRangeHigh   longitude  latitude  \
0  548932.8      508107.94       589757.66 -121.958287  37.39582   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [937]:
type(sale_estimate_pdf)
sale_estimate_pdf.head

<bound method NDFrame.head of       price  priceRangeLow  priceRangeHigh   longitude  latitude  \
0  548932.8      508107.94       589757.66 -121.958287  37.39582   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [938]:
#95110
url = "https://realty-mole-property-api.p.rapidapi.com/salePrice"

querystring = {"compCount":"10","bathrooms":"1","address":"175 W St James St, San Jose, CA","bedrooms":"1","propertyType":"Condo"}

headers = {
    'x-rapidapi-key': "ca0f4a0c63msheb56ce445093e55p1d9232jsn4658d02eedac",
    'x-rapidapi-host': "realty-mole-property-api.p.rapidapi.com"
    }

response1 = requests.request("GET", url, headers=headers, params=querystring)

print(response1.text)
sale_estimate_pdf_1 = json_normalize(response1.json())
sale_estimate_pdf_1.head()

{"price":609200,"priceRangeLow":571844.05,"priceRangeHigh":646555.95,"longitude":-121.895531,"latitude":37.3379369,"listings":[{"id":"175-W-Saint-James-St,-Unit-1004,-San-Jose,-CA-95110","formattedAddress":"175 W Saint James St, Unit 1004, San Jose, CA 95110","longitude":-121.895693,"latitude":37.338052,"city":"San Jose","state":"CA","zipcode":"95110","price":579000,"publishedDate":"2021-04-20T09:39:13.652Z","distance":0.01922881400750797,"daysOld":27.9,"correlation":0.9995,"address":"175 W Saint James St, Unit 1004","county":"Santa Clara County","bedrooms":1,"bathrooms":1,"propertyType":"Condo","squareFootage":904},{"id":"175-W-Saint-James-St,-Unit-605,-San-Jose,-CA-95110","formattedAddress":"175 W Saint James St, Unit 605, San Jose, CA 95110","longitude":-121.895693,"latitude":37.338052,"city":"San Jose","state":"CA","zipcode":"95110","price":580000,"publishedDate":"2020-12-15T10:49:25.875Z","distance":0.01922881400750797,"daysOld":153.85,"correlation":0.9989,"address":"175 W Saint J

,price,priceRangeLow,priceRangeHigh,longitude,latitude,listings
0,609200,571844.05,646555.95,-121.895531,37.337937,"[{'id': '175-W-Saint-James-St,-Unit-1004,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 1004, San Jose, CA 95110', 'longitude': -121.895693, 'latitude': 37.338052, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 579000, 'publishedDate': '2021-04-20T09:39:13.652Z', 'distance': 0.01922881400750797, 'daysOld': 27.9, 'correlation': 0.9995, 'address': '175 W Saint James St, Unit 1004', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 904}, {'id': '175-W-Saint-James-St,-Unit-605,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 605, San Jose, CA 95110', 'longitude': -121.895693, 'latitude': 37.338052, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 580000, 'publishedDate': '2020-12-15T10:49:25.875Z', 'distance': 0.01922881400750797, 'daysOld': 153.85, 'correlation': 0.9989, 'address': '175 W Saint James St, Unit 605', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 904}, {'id': '175-W-Saint-James-St,-Unit-1204,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 1204, San Jose, CA 95110', 'longitude': -121.895693, 'latitude': 37.338052, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 575000, 'publishedDate': '2020-12-08T11:44:17.753Z', 'distance': 0.01922881400750797, 'daysOld': 160.81, 'correlation': 0.9989, 'address': '175 W Saint James St, Unit 1204', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 904}, {'id': '188-W-Saint-James-St,-Unit-10314,-San-Jose,-CA-95110', 'formattedAddress': '188 W Saint James St, Unit 10314, San Jose, CA 95110', 'longitude': -121.89577, 'latitude': 37.337511, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 679000, 'publishedDate': '2021-03-10T11:36:15.902Z', 'distance': 0.051915588762889965, 'daysOld': 68.82, 'correlation': 0.9985, 'address': '188 W Saint James St, Unit 10314', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 742}, {'id': '175-W-Saint-James-St,-Unit-807,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 807, San Jose, CA 95110', 'longitude': -121.895733, 'latitude': 37.338237, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 633000, 'publishedDate': '2020-11-27T12:04:15.851Z', 'distance': 0.037889921340094286, 'daysOld': 171.8, 'correlation': 0.9984, 'address': '175 W Saint James St, Unit 807', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 663}]"


In [939]:
#94085
url = "https://realty-mole-property-api.p.rapidapi.com/salePrice"

querystring = {"compCount":"10","bathrooms":"1","address":"612 Arcadia Ter #102 SUNNYVALE, CA","bedrooms":"1","propertyType":"Condo"}

headers = {
    'x-rapidapi-key': "ca0f4a0c63msheb56ce445093e55p1d9232jsn4658d02eedac",
    'x-rapidapi-host': "realty-mole-property-api.p.rapidapi.com"
    }

response2 = requests.request("GET", url, headers=headers, params=querystring)

print(response2.text)
sale_estimate_pdf_2 = json_normalize(response2.json())
sale_estimate_pdf_2.head()

{"price":685559.8,"priceRangeLow":658265.59,"priceRangeHigh":712854.01,"longitude":-122.0117097,"latitude":37.3929436,"listings":[{"id":"615-Arcadia-Ter,-Unit-102,-Sunnyvale,-CA-94085","formattedAddress":"615 Arcadia Ter, Unit 102, Sunnyvale, CA 94085","longitude":-122.011949,"latitude":37.392223,"city":"Sunnyvale","state":"CA","zipcode":"94085","price":729000,"publishedDate":"2020-11-12T15:20:13.767Z","distance":0.08296135960565154,"daysOld":186.66,"correlation":0.9973,"address":"615 Arcadia Ter, Unit 102","county":"Santa Clara County","bedrooms":1,"bathrooms":1,"propertyType":"Condo","squareFootage":867},{"id":"607-Arcadia-Ter,-Unit-102,-Sunnyvale,-CA-94085","formattedAddress":"607 Arcadia Ter, Unit 102, Sunnyvale, CA 94085","longitude":-122.012935,"latitude":37.392414,"city":"Sunnyvale","state":"CA","zipcode":"94085","price":699999,"publishedDate":"2021-04-20T10:07:57.125Z","distance":0.12336626993753262,"daysOld":27.88,"correlation":0.9971,"address":"607 Arcadia Ter, Unit 102","cou

,price,priceRangeLow,priceRangeHigh,longitude,latitude,listings
0,685559.8,658265.59,712854.01,-122.01171,37.392944,"[{'id': '615-Arcadia-Ter,-Unit-102,-Sunnyvale,-CA-94085', 'formattedAddress': '615 Arcadia Ter, Unit 102, Sunnyvale, CA 94085', 'longitude': -122.011949, 'latitude': 37.392223, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94085', 'price': 729000, 'publishedDate': '2020-11-12T15:20:13.767Z', 'distance': 0.08296135960565154, 'daysOld': 186.66, 'correlation': 0.9973, 'address': '615 Arcadia Ter, Unit 102', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 867}, {'id': '607-Arcadia-Ter,-Unit-102,-Sunnyvale,-CA-94085', 'formattedAddress': '607 Arcadia Ter, Unit 102, Sunnyvale, CA 94085', 'longitude': -122.012935, 'latitude': 37.392414, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94085', 'price': 699999, 'publishedDate': '2021-04-20T10:07:57.125Z', 'distance': 0.12336626993753262, 'daysOld': 27.88, 'correlation': 0.9971, 'address': '607 Arcadia Ter, Unit 102', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 867}, {'id': '708-San-Conrado-Ter,-Unit-2,-Sunnyvale,-CA-94085', 'formattedAddress': '708 San Conrado Ter, Unit 2, Sunnyvale, CA 94085', 'longitude': -122.015528, 'latitude': 37.392516, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94085', 'price': 648800, 'publishedDate': '2020-09-23T13:42:33.655Z', 'distance': 0.34103590195357847, 'daysOld': 236.73, 'correlation': 0.9914, 'address': '708 San Conrado Ter, Unit 2', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 784}, {'id': '767-N-Fair-Oaks-Ave,-Apt-4,-Sunnyvale,-CA-94085', 'formattedAddress': '767 N Fair Oaks Ave, Apt 4, Sunnyvale, CA 94085', 'longitude': -122.015741, 'latitude': 37.393046, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94085', 'price': 695000, 'publishedDate': '2021-05-16T13:52:08.994Z', 'distance': 0.3567167346926699, 'daysOld': 1.73, 'correlation': 0.8948, 'address': '767 N Fair Oaks Ave, Apt 4', 'county': 'Santa Clara County', 'bedrooms': 2, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 926}, {'id': '785-N-Fair-Oaks-Ave,-Apt-3,-Sunnyvale,-CA-94085', 'formattedAddress': '785 N Fair Oaks Ave, Apt 3, Sunnyvale, CA 94085', 'longitude': -122.015621, 'latitude': 37.393341, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94085', 'price': 655000, 'publishedDate': '2021-02-22T12:17:54.743Z', 'distance': 0.34873811085723844, 'daysOld': 84.79, 'correlation': 0.8947, 'address': '785 N Fair Oaks Ave, Apt 3', 'county': 'Santa Clara County', 'bedrooms': 2, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 926}]"


In [940]:
#  95051
url = "https://realty-mole-property-api.p.rapidapi.com/salePrice"

querystring = {"compCount":"10","bathrooms":"1","address":"150 Saratoga Ave #318 SANTA CLARA, CA","bedrooms":"1","propertyType":"Condo"}

headers = {
    'x-rapidapi-key': "ca0f4a0c63msheb56ce445093e55p1d9232jsn4658d02eedac",
    'x-rapidapi-host': "realty-mole-property-api.p.rapidapi.com"
    }

response3 = requests.request("GET", url, headers=headers, params=querystring)

print(response3.text)
sale_estimate_pdf_3 = json_normalize(response3.json())
sale_estimate_pdf_3.head()

{"price":545990,"priceRangeLow":517242.24,"priceRangeHigh":574737.76,"longitude":-121.9692176,"latitude":37.3265099,"listings":[{"id":"150-Saratoga-Ave,-Unit-318,-Santa-Clara,-CA-95051","formattedAddress":"150 Saratoga Ave, Unit 318, Santa Clara, CA 95051","longitude":-121.969308,"latitude":37.326738,"city":"Santa Clara","state":"CA","zipcode":"95051","price":598000,"publishedDate":"2021-03-01T09:42:36.168Z","distance":0.026622923377438622,"daysOld":77.9,"correlation":0.9991,"address":"150 Saratoga Ave, Unit 318","county":"Santa Clara County","bedrooms":1,"bathrooms":1,"propertyType":"Condo","squareFootage":599},{"id":"121-Buckingham-Dr,-Unit-64,-Santa-Clara,-CA-95051","formattedAddress":"121 Buckingham Dr, Unit 64, Santa Clara, CA 95051","longitude":-121.97017,"latitude":37.326175,"city":"Santa Clara","state":"CA","zipcode":"95051","price":499950,"publishedDate":"2020-10-21T10:19:41.661Z","distance":0.09218162987359514,"daysOld":208.87,"correlation":0.997,"address":"121 Buckingham Dr,

,price,priceRangeLow,priceRangeHigh,longitude,latitude,listings
0,545990,517242.24,574737.76,-121.969218,37.32651,"[{'id': '150-Saratoga-Ave,-Unit-318,-Santa-Clara,-CA-95051', 'formattedAddress': '150 Saratoga Ave, Unit 318, Santa Clara, CA 95051', 'longitude': -121.969308, 'latitude': 37.326738, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95051', 'price': 598000, 'publishedDate': '2021-03-01T09:42:36.168Z', 'distance': 0.026622923377438622, 'daysOld': 77.9, 'correlation': 0.9991, 'address': '150 Saratoga Ave, Unit 318', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 599}, {'id': '121-Buckingham-Dr,-Unit-64,-Santa-Clara,-CA-95051', 'formattedAddress': '121 Buckingham Dr, Unit 64, Santa Clara, CA 95051', 'longitude': -121.97017, 'latitude': 37.326175, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95051', 'price': 499950, 'publishedDate': '2020-10-21T10:19:41.661Z', 'distance': 0.09218162987359514, 'daysOld': 208.87, 'correlation': 0.997, 'address': '121 Buckingham Dr, Unit 64', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 656}, {'id': '151-Buckingham-Dr,-Unit-223,-Santa-Clara,-CA-95051', 'formattedAddress': '151 Buckingham Dr, Unit 223, Santa Clara, CA 95051', 'longitude': -121.970891, 'latitude': 37.326826, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95051', 'price': 548000, 'publishedDate': '2021-05-16T13:10:11.074Z', 'distance': 0.15225107164981688, 'daysOld': 1.75, 'correlation': 0.9966, 'address': '151 Buckingham Dr, Unit 223', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 661}, {'id': '121-Buckingham-Dr,-Unit-40,-Santa-Clara,-CA-95051', 'formattedAddress': '121 Buckingham Dr, Unit 40, Santa Clara, CA 95051', 'longitude': -121.970785, 'latitude': 37.325857, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95051', 'price': 549000, 'publishedDate': '2021-05-16T13:10:11.066Z', 'distance': 0.1566303539545646, 'daysOld': 1.75, 'correlation': 0.9965, 'address': '121 Buckingham Dr, Unit 40', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 656}, {'id': '151-Buckingham-Dr,-Unit-268,-Santa-Clara,-CA-95051', 'formattedAddress': '151 Buckingham Dr, Unit 268, Santa Clara, CA 95051', 'longitude': -121.971215, 'latitude': 37.327009, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95051', 'price': 535000, 'publishedDate': '2020-12-09T10:53:02.056Z', 'distance': 0.18533299940063996, 'daysOld': 159.85, 'correlation': 0.9952, 'address': '151 Buckingham Dr, Unit 268', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 770}]"


In [941]:
#  94086
url = "https://realty-mole-property-api.p.rapidapi.com/salePrice"

querystring = {"compCount":"10","bathrooms":"1","address":"420 E Evelyn Ave Unit Z102 SUNNYVALE, CA","bedrooms":"1","propertyType":"Condo"}

headers = {
    'x-rapidapi-key': "ca0f4a0c63msheb56ce445093e55p1d9232jsn4658d02eedac",
    'x-rapidapi-host': "realty-mole-property-api.p.rapidapi.com"
    }

response4 = requests.request("GET", url, headers=headers, params=querystring)

print(response4.text)

sale_estimate_pdf_4 = json_normalize(response4.json())
sale_estimate_pdf_4.head()

{"price":869177.6,"priceRangeLow":707107.29,"priceRangeHigh":1031247.91,"longitude":-122.0252971,"latitude":37.3756436,"listings":[{"id":"420-E-Evelyn-Ave,-Unit-Z102,-Sunnyvale,-CA-94086","formattedAddress":"420 E Evelyn Ave, Unit Z102, Sunnyvale, CA 94086","longitude":-122.025276,"latitude":37.375621,"city":"Sunnyvale","state":"CA","zipcode":"94086","price":699888,"publishedDate":"2021-03-05T11:14:24.440Z","distance":0.003132616579109452,"daysOld":73.84,"correlation":0.9996,"address":"420 E Evelyn Ave, Unit Z102","county":"Santa Clara County","bedrooms":1,"bathrooms":1,"propertyType":"Condo","squareFootage":605},{"id":"610-S-Fair-Oaks-Ave,-Sunnyvale,-CA-94086","formattedAddress":"610 S Fair Oaks Ave, Sunnyvale, CA 94086","longitude":-122.023038,"latitude":37.366644,"city":"Sunnyvale","state":"CA","zipcode":"94086","price":899000,"publishedDate":"2021-02-22T13:19:34.101Z","distance":1.0215655390968268,"daysOld":84.75,"correlation":0.8464,"address":"610 S Fair Oaks Ave","county":"Santa 

,price,priceRangeLow,priceRangeHigh,longitude,latitude,listings
0,869177.6,707107.29,1031247.91,-122.025297,37.375644,"[{'id': '420-E-Evelyn-Ave,-Unit-Z102,-Sunnyvale,-CA-94086', 'formattedAddress': '420 E Evelyn Ave, Unit Z102, Sunnyvale, CA 94086', 'longitude': -122.025276, 'latitude': 37.375621, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94086', 'price': 699888, 'publishedDate': '2021-03-05T11:14:24.440Z', 'distance': 0.003132616579109452, 'daysOld': 73.84, 'correlation': 0.9996, 'address': '420 E Evelyn Ave, Unit Z102', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 605}, {'id': '610-S-Fair-Oaks-Ave,-Sunnyvale,-CA-94086', 'formattedAddress': '610 S Fair Oaks Ave, Sunnyvale, CA 94086', 'longitude': -122.023038, 'latitude': 37.366644, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94086', 'price': 899000, 'publishedDate': '2021-02-22T13:19:34.101Z', 'distance': 1.0215655390968268, 'daysOld': 84.75, 'correlation': 0.8464, 'address': '610 S Fair Oaks Ave', 'county': 'Santa Clara County', 'bedrooms': 2, 'bathrooms': 1.5, 'propertyType': 'Condo', 'squareFootage': 1122}, {'id': '260-Ficus-Ter,-Sunnyvale,-CA-94086', 'formattedAddress': '260 Ficus Ter, Sunnyvale, CA 94086', 'longitude': -122.019501, 'latitude': 37.372503, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94086', 'price': 1199000, 'publishedDate': '2020-12-08T14:35:31.777Z', 'distance': 0.6205915031944449, 'daysOld': 160.7, 'correlation': 0.8216, 'address': '260 Ficus Ter', 'county': 'Santa Clara County', 'bedrooms': 2, 'bathrooms': 2, 'propertyType': 'Condo', 'squareFootage': 1276}, {'id': '250-Santa-Fe-Ter,-Apt-226,-Sunnyvale,-CA-94085', 'formattedAddress': '250 Santa Fe Ter, Apt 226, Sunnyvale, CA 94085', 'longitude': -122.016812, 'latitude': 37.379934, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94085', 'price': 750000, 'publishedDate': '2021-04-20T11:12:37.771Z', 'distance': 0.8896551043998654, 'daysOld': 27.84, 'correlation': 0.8162, 'address': '250 Santa Fe Ter, Apt 226', 'county': 'Santa Clara County', 'bedrooms': 2, 'bathrooms': 2, 'propertyType': 'Condo', 'squareFootage': 1061}, {'id': '250-Santa-Fe-Ter,-Apt-222,-Sunnyvale,-CA-94085', 'formattedAddress': '250 Santa Fe Ter, Apt 222, Sunnyvale, CA 94085', 'longitude': -122.016995, 'latitude': 37.380306, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94085', 'price': 798000, 'publishedDate': '2021-04-20T11:12:37.773Z', 'distance': 0.899285750864831, 'daysOld': 27.84, 'correlation': 0.816, 'address': '250 Santa Fe Ter, Apt 222', 'county': 'Santa Clara County', 'bedrooms': 2, 'bathrooms': 2, 'propertyType': 'Condo', 'squareFootage': 1061}]"


In [942]:
#  94087

url = "https://realty-mole-property-api.p.rapidapi.com/salePrice"

querystring = {"compCount":"10","bathrooms":"1","address":"880 E Fremont Ave #304, SUNNYVALE, CA","bedrooms":"1","propertyType":"Condo"}

headers = {
    'x-rapidapi-key': "ca0f4a0c63msheb56ce445093e55p1d9232jsn4658d02eedac",
    'x-rapidapi-host': "realty-mole-property-api.p.rapidapi.com"
    }

response5 = requests.request("GET", url, headers=headers, params=querystring)

print(response5.text)
sale_estimate_pdf_5 = json_normalize(response5.json())
sale_estimate_pdf_5.head()

{"price":591777.6,"priceRangeLow":541014.47,"priceRangeHigh":642540.73,"longitude":-122.0154158,"latitude":37.351578,"listings":[{"id":"880-E-Fremont-Ave,-Apt-304,-Sunnyvale,-CA-94087","formattedAddress":"880 E Fremont Ave, Apt 304, Sunnyvale, CA 94087","longitude":-122.015425,"latitude":37.351275,"city":"Sunnyvale","state":"CA","zipcode":"94087","price":675000,"publishedDate":"2021-05-16T13:52:08.981Z","distance":0.03373943352362619,"daysOld":1.73,"correlation":0.9992,"address":"880 E Fremont Ave, Apt 304","county":"Santa Clara County","bedrooms":1,"bathrooms":1,"propertyType":"Condo","squareFootage":733},{"id":"880-E-Fremont-Ave,-Apt-325,-Sunnyvale,-CA-94087","formattedAddress":"880 E Fremont Ave, Apt 325, Sunnyvale, CA 94087","longitude":-122.015577,"latitude":37.351354,"city":"Sunnyvale","state":"CA","zipcode":"94087","price":598000,"publishedDate":"2021-02-05T10:33:41.930Z","distance":0.028727274102486308,"daysOld":101.86,"correlation":0.9989,"address":"880 E Fremont Ave, Apt 325"

,price,priceRangeLow,priceRangeHigh,longitude,latitude,listings
0,591777.6,541014.47,642540.73,-122.015416,37.351578,"[{'id': '880-E-Fremont-Ave,-Apt-304,-Sunnyvale,-CA-94087', 'formattedAddress': '880 E Fremont Ave, Apt 304, Sunnyvale, CA 94087', 'longitude': -122.015425, 'latitude': 37.351275, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94087', 'price': 675000, 'publishedDate': '2021-05-16T13:52:08.981Z', 'distance': 0.03373943352362619, 'daysOld': 1.73, 'correlation': 0.9992, 'address': '880 E Fremont Ave, Apt 304', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 733}, {'id': '880-E-Fremont-Ave,-Apt-325,-Sunnyvale,-CA-94087', 'formattedAddress': '880 E Fremont Ave, Apt 325, Sunnyvale, CA 94087', 'longitude': -122.015577, 'latitude': 37.351354, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94087', 'price': 598000, 'publishedDate': '2021-02-05T10:33:41.930Z', 'distance': 0.028727274102486308, 'daysOld': 101.86, 'correlation': 0.9989, 'address': '880 E Fremont Ave, Apt 325', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 713}, {'id': '880-E-Fremont-Ave,-Apt-414,-Sunnyvale,-CA-94087', 'formattedAddress': '880 E Fremont Ave, Apt 414, Sunnyvale, CA 94087', 'longitude': -122.015975, 'latitude': 37.35159, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94087', 'price': 598000, 'publishedDate': '2021-05-16T13:51:56.806Z', 'distance': 0.04950187011625293, 'daysOld': 1.73, 'correlation': 0.9989, 'address': '880 E Fremont Ave, Apt 414', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 733}, {'id': '880-E-Fremont-Ave,-Apt-205,-Sunnyvale,-CA-94087', 'formattedAddress': '880 E Fremont Ave, Apt 205, Sunnyvale, CA 94087', 'longitude': -122.015807, 'latitude': 37.351898, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94087', 'price': 499888, 'publishedDate': '2021-05-16T13:10:11.058Z', 'distance': 0.04967184455934464, 'daysOld': 1.75, 'correlation': 0.9989, 'address': '880 E Fremont Ave, Apt 205', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 733}, {'id': '880-E-Fremont-Ave,-Apt-508,-Sunnyvale,-CA-94087', 'formattedAddress': '880 E Fremont Ave, Apt 508, Sunnyvale, CA 94087', 'longitude': -122.016257, 'latitude': 37.351509, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94087', 'price': 588000, 'publishedDate': '2021-05-16T13:51:56.804Z', 'distance': 0.07483341930801804, 'daysOld': 1.73, 'correlation': 0.9983, 'address': '880 E Fremont Ave, Apt 508', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 712}]"


#### Combining 6 data frames retrieved from all 6 API calls for 6 different zip codes

In [943]:
sale_estimate_pdfs = pd.concat([sale_estimate_pdf, sale_estimate_pdf_1, sale_estimate_pdf_2, sale_estimate_pdf_3, sale_estimate_pdf_4, sale_estimate_pdf_5], ignore_index=True)
sale_estimate_pdfs

,price,priceRangeLow,priceRangeHigh,longitude,latitude,listings
0,548932.8,508107.94,589757.66,-121.958287,37.395820,"[{'id': '1883-Agnew-Rd,-Unit-247,-Santa-Clara,-CA-95054', 'formattedAddress': '1883 Agnew Rd, Unit 247, Santa Clara, CA 95054', 'longitude': -121.958622, 'latitude': 37.395726, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 598888, 'publishedDate': '2020-12-28T11:26:59.931Z', 'distance': 0.03139360151479152, 'daysOld': 140.83, 'correlation': 0.9989, 'address': '1883 Agnew Rd, Unit 247', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 870}, {'id': '2200-Agnew-Rd,-Apt-309,-Santa-Clara,-CA-95054', 'formattedAddress': '2200 Agnew Rd, Apt 309, Santa Clara, CA 95054', 'longitude': -121.96357, 'latitude': 37.392453, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 499000, 'publishedDate': '2021-05-16T13:10:11.218Z', 'distance': 0.5989462621367421, 'daysOld': 1.75, 'correlation': 0.9797, 'address': '2200 Agnew Rd, Apt 309', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 816}, {'id': '2200-Agnew-Rd,-Apt-220,-Santa-Clara,-CA-95054', 'formattedAddress': '2200 Agnew Rd, Apt 220, Santa Clara, CA 95054', 'longitude': -121.963536, 'latitude': 37.392146, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 548888, 'publishedDate': '2021-02-13T09:27:42.910Z', 'distance': 0.6186429069456735, 'daysOld': 93.91, 'correlation': 0.979, 'address': '2200 Agnew Rd, Apt 220', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 816}, {'id': '2200-Agnew-Rd,-Apt-207,-Santa-Clara,-CA-95054', 'formattedAddress': '2200 Agnew Rd, Apt 207, Santa Clara, CA 95054', 'longitude': -121.963738, 'latitude': 37.392387, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 599000, 'publishedDate': '2020-12-28T11:26:59.930Z', 'distance': 0.6151440061754975, 'daysOld': 140.83, 'correlation': 0.9789, 'address': '2200 Agnew Rd, Apt 207', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 816}, {'id': '2200-Agnew-Rd,-Apt-106,-Santa-Clara,-CA-95054', 'formattedAddress': '2200 Agnew Rd, Apt 106, Santa Clara, CA 95054', 'longitude': -121.963824, 'latitude': 37.392354, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 498888, 'publishedDate': '2020-12-28T08:51:41.622Z', 'distance': 0.6233892631156447, 'daysOld': 140.93, 'correlation': 0.9787, 'address': '2200 Agnew Rd, Apt 106', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 816}]"
1,609200.0,571844.05,646555.95,-121.895531,37.337937,"[{'id': '175-W-Saint-James-St,-Unit-1004,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 1004, San Jose, CA 95110', 'longitude': -121.895693, 'latitude': 37.338052, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 579000, 'publishedDate': '2021-04-20T09:39:13.652Z', 'distance': 0.01922881400750797, 'daysOld': 27.9, 'correlation': 0.9995, 'address': '175 W Saint James St, Unit 1004', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 904}, {'id': '175-W-Saint-James-St,-Unit-605,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 605, San Jose, CA 95110', 'longitude': -121.895693, 'latitude': 37.338052, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 580000, 'publishedDate': '2020-12-15T10:49:25.875Z', 'distance': 0.01922881400750797, 'daysOld': 153.85, 'correlation': 0.9989, 'address': '175 W Saint James St, Unit 605', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 904}, {'id': '175-W-Saint-James-St,-Unit-1204,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 1204, San Jose, CA 95110', 'longitude': -121.895693, 'latitude': 37.338

In [944]:
# sale_estimate_pdfs.listings[0][0]

In [945]:
# range(len(sale_estimate_pdfs.listings))

In [946]:
# pytz.all_timezones

In [947]:
datetime_str = sale_estimate_pdfs.listings[0][0]['publishedDate']
print(datetime_str)
date_time_obj = dt.datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S.%fZ')
# timezone = pytz.timezone('America/Los_Angeles')
# print(timezone_date_time_obj.tzinfo)
# timezone_date_time_obj = timezone.localize(date_time_obj)



print(date_time_obj.date())
# print(date_time_obj.tzinfo)
# print(timezone_date_time_obj)

2020-12-28T11:26:59.931Z
2020-12-28


In [948]:
# publishedDate = maya.parse(sale_estimate_pdf.listings[0][0]['publishedDate']).datetime(to_timezone='America/Los_Angeles', naive=False)
# print(publishedDate.date())

In [949]:
# type(sale_estimate_pdf.listings[0][0]['publishedDate'])

In [950]:
# sale_estimate_pdf.listings[0][0]['price']

In [951]:
# sale_estimate_pdf.listings[0][1]['publishedDate']

In [952]:
# sale_estimate_pdf.listings[0][1]['price']

In [953]:
# sale_estimate_pdf.listings[0][4]['zipcode']

In [954]:
# len(sale_estimate_pdf.listings[0])

In [955]:
# sale_estimate_pdf.listings[0]

In [956]:
# Definte CA Pandemic Period

pandemic_dt1 = dt.date(2020, 3, 18)
pandemic_dt2 = date.today()

print(pandemic_dt1)
print(pandemic_dt2)

2020-03-18
2021-05-18


In [957]:
# construct a data zip that can be summarized at zip code level. Restrict to all transactions since Mar 18, 2020

columns = ['publishedDate', 'price', 'squareFootage', 'sq_Footage_price', 'zipcode']
main_df = pd.DataFrame(columns=columns)
counter = 0
for k in range(len(sale_estimate_pdfs.listings)):
    for i in range(len(sale_estimate_pdfs.listings[k])):
        publishedDate = maya.parse(sale_estimate_pdfs.listings[k][i]['publishedDate']).datetime(to_timezone='America/Los_Angeles', naive=False)
    #     print(publishedDate.date())
        if publishedDate.date() in DatetimeRange(pandemic_dt1, pandemic_dt2):
            print(counter)
            main_df.loc[counter, 'publishedDate'] = publishedDate.date()
            print(publishedDate.date())
            main_df.loc[counter, 'price'] = sale_estimate_pdfs.listings[k][i]['price']
            print( sale_estimate_pdfs.listings[k][i]['price'])
            main_df.loc[counter, 'squareFootage'] =  sale_estimate_pdfs.listings[k][i]['squareFootage']
            print(sale_estimate_pdfs.listings[k][i]['squareFootage'])
            main_df.loc[counter, 'sq_Footage_price'] =  round(sale_estimate_pdfs.listings[k][i]['price'] / sale_estimate_pdfs.listings[k][i]['squareFootage'],0)
            print(round(sale_estimate_pdfs.listings[k][i]['price'] / sale_estimate_pdfs.listings[k][i]['squareFootage'],0))
            main_df.loc[counter, 'zipcode'] =  sale_estimate_pdfs.listings[k][i]['zipcode']
            print(sale_estimate_pdfs.listings[k][i]['zipcode'])
            counter+=1

0
2020-12-28
598888
870
688.0
95054
1
2021-05-16
499000
816
612.0
95054
2
2021-02-13
548888
816
673.0
95054
3
2020-12-28
599000
816
734.0
95054
4
2020-12-28
498888
816
611.0
95054
5
2021-04-20
579000
904
640.0
95110
6
2020-12-15
580000
904
642.0
95110
7
2020-12-08
575000
904
636.0
95110
8
2021-03-10
679000
742
915.0
95110
9
2020-11-27
633000
663
955.0
95110
10
2020-11-12
729000
867
841.0
94085
11
2021-04-20
699999
867
807.0
94085
12
2020-09-23
648800
784
828.0
94085
13
2021-05-16
695000
926
751.0
94085
14
2021-02-22
655000
926
707.0
94085
15
2021-03-01
598000
599
998.0
95051
16
2020-10-21
499950
656
762.0
95051
17
2021-05-16
548000
661
829.0
95051
18
2021-05-16
549000
656
837.0
95051
19
2020-12-09
535000
770
695.0
95051
20
2021-03-05
699888
605
1157.0
94086
21
2021-02-22
899000
1122
801.0
94086
22
2020-12-08
1199000
1276
940.0
94086
23
2021-04-20
750000
1061
707.0
94085
24
2021-04-20
798000
1061
752.0
94085
25
2021-05-16
675000
733
921.0
94087
26
2021-02-05
598000
713
839.0
94087
27
20

In [958]:
# sale_estimate.listings[0][0]

In [959]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   publishedDate     30 non-null     object
 1   price             30 non-null     object
 2   squareFootage     30 non-null     object
 3   sq_Footage_price  30 non-null     object
 4   zipcode           30 non-null     object
dtypes: object(5)
memory usage: 2.5+ KB


In [960]:
main_df.price = main_df.fillna(0).price.astype(int)
main_df.squareFootage =  main_df.fillna(0).squareFootage.astype(int)
main_df.sq_Footage_price = main_df.fillna(0).sq_Footage_price.astype(float)


In [961]:
main_df

,publishedDate,price,squareFootage,sq_Footage_price,zipcode
0,2020-12-28,598888,870,688.0,95054
1,2021-05-16,499000,816,612.0,95054
2,2021-02-13,548888,816,673.0,95054
3,2020-12-28,599000,816,734.0,95054
4,2020-12-28,498888,816,611.0,95054
5,2021-04-20,579000,904,640.0,95110
6,2020-12-15,580000,904,642.0,95110
7,2020-12-08,575000,904,636.0,95110
8,2021-03-10,679000,742,915.0,95110
9,2020-11-27,633000,663,955.0,95110


In [962]:
geoz_group = main_df.groupby('zipcode').agg({'price': np.median, 'sq_Footage_price': np.median})


In [963]:
geoz_group

,price,sq_Footage_price
zipcode,,
94085,699999,752.0
94086,899000,940.0
94087,598000,826.0
95051,548000,829.0
95054,548888,673.0
95110,580000,642.0


In [964]:
# don't need to re-run
cols = list(pd.read_csv("Self_Created_DB.csv", nrows =1))
self_created_db = pd.read_csv("Self_Created_DB.csv", usecols =[i for i in cols if i != 'Index'])

In [965]:
self_created_db

,Address,Sold Price,Listed Price,Redfin Estimate,As of,Price / sq ft,Days on Redfin,Sold Date,Is it 1-bed/1-bath,Is it zip-code 95054,is it in the city of Santa Clara,is it in Santa Clara County,is it in San Mateo County,Sold Price Pre-Pandemic
0,"1883 Agnew Rd #458 SANTA CLARA, CA 95054","$641,000","$699,000","$697,970",8/11/2020,$737,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"3901 Lick Mill Blvd #410 SANTA CLARA, CA 95054","$810,000","$815,000","$815,579",8/11/2020,$900,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"1883 Agnew Rd #460 SANTA CLARA, CA 95054","$873,000","$799,999","$822,983",8/11/2020,$816,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"613 Arcadia Ter #105 SUNNYVALE, CA 94085","$788,000","$710,000","$785,429",8/23/2020,$909,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"435 Milan Dr #201 SAN JOSE, CA 95134","$679,000","$679,000","$684,121",8/23/2020,$780,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"666 Encore Way SAN JOSE, CA 95134","$625,000","$675,000","$660,933",8/23/2020,$805,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"999 W Evelyn Ter #64 Sunnyvale, CA 94086","$658,000","$668,000",NaN,8/29/2020,$800,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"1883 Agnew Rd #420 SANTA CLARA, CA 95054","$856,000","$838,800","$846,786",9/8/2020,$800,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"1883 Agnew Rd #350 Santa Clara, CA 95054","$820,000","$798,000","$809,405",9/11/2020,$712,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"3901 Lick Mill Blvd #305 Santa Clara, CA 95054",NaN,"$728,000","$725,484",5/16/2021,$809,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [966]:
# main_df2 = self_created_db.loc[(self_created_db['Is it 1-bed/1-bath']=='Y') & (self_created_db['Is it zip-code 95054']=='Y'), ['Price / sq ft', 'Sold Price',  'Sold Price Pre-Pandemic']]

In [967]:
# main_df2['zipcode'] = '95054'

In [968]:
# making backup copy of main_df2
# main_df2_bkp = main_df2.copy()
# main_df2_bkp

In [969]:
main_df2 = self_created_db.loc[self_created_db['Is it 1-bed/1-bath']=='Y' , ['Address','Price / sq ft', 'Sold Price',  'Sold Price Pre-Pandemic']]

In [970]:
main_df2['zipcode'] = main_df2['Address'].str[-5:]


In [971]:
main_df2

,Address,Price / sq ft,Sold Price,Sold Price Pre-Pandemic,zipcode
14,"175 W Saint James St #807, San Jose, CA 95110",$955,"$620,000",NaN,95110
18,"612 Arcadia Ter #102 SUNNYVALE, CA 94085",$963,"$835,000",NaN,94085
19,"604 Arcadia Ter #103 SUNNYVALE, CA 94085",$924,"$801,000",NaN,94085
20,"3901 Lick Mill Blvd #438 SANTA CLARA, CA 95054","$1,024","$930,000",NaN,95054
21,"3901 Lick Mill Blvd #301 SANTA CLARA, CA 95054",$910,"$787,000",NaN,95054
22,"3901 Lick Mill Blvd #259 SANTA CLARA, CA 95054","$1,020","$882,000",NaN,95054
23,"3901 Lick Mill Blvd #337 SANTA CLARA, CA 95054",$936,"$850,000",NaN,95054
24,"3901 Lick Mill Blvd #405 SANTA CLARA, CA 95054",$894,"$805,000",NaN,95054
25,"3901 Lick Mill Blvd #443 SANTA CLARA, CA 95054",$925,"$840,000",NaN,95054
31,"150 Saratoga Ave #318 SANTA CLARA, CA 95051",$998,"$650,000",NaN,95051


In [972]:
# dropping Address
main_df2.drop('Address', inplace=True, axis=1)

In [973]:
# price / sq ft needs to be changed to float
main_df2[main_df2.columns[0,]] = main_df2[main_df2.columns[0,]].replace('[\$,]', '', regex=True).astype(float)


In [974]:
main_df2

,Price / sq ft,Sold Price,Sold Price Pre-Pandemic,zipcode
14,955.0,"$620,000",NaN,95110
18,963.0,"$835,000",NaN,94085
19,924.0,"$801,000",NaN,94085
20,1024.0,"$930,000",NaN,95054
21,910.0,"$787,000",NaN,95054
22,1020.0,"$882,000",NaN,95054
23,936.0,"$850,000",NaN,95054
24,894.0,"$805,000",NaN,95054
25,925.0,"$840,000",NaN,95054
31,998.0,"$650,000",NaN,95051


In [975]:
main_df2['Sold Price Pre-Pandemic'] = main_df2['Sold Price Pre-Pandemic'].fillna(0)
main_df2['Sold Price'] = main_df2['Sold Price'].fillna(0)

In [976]:
main_df2

,Price / sq ft,Sold Price,Sold Price Pre-Pandemic,zipcode
14,955.0,"$620,000",0,95110
18,963.0,"$835,000",0,94085
19,924.0,"$801,000",0,94085
20,1024.0,"$930,000",0,95054
21,910.0,"$787,000",0,95054
22,1020.0,"$882,000",0,95054
23,936.0,"$850,000",0,95054
24,894.0,"$805,000",0,95054
25,925.0,"$840,000",0,95054
31,998.0,"$650,000",0,95051


In [977]:
#  main_df2['Sold Price Pre-Pandemic'] != 0

In [978]:
# main_df2['Sold Price Pre-Pandemic'] is None

In [979]:
main_df2['Sold Price'] = main_df2.apply(lambda x: x[1] if x[2] == 0 else x[2], axis = 1)

In [980]:
# same thing to sold price
main_df2[main_df2.columns[1:3,]] = main_df2[main_df2.columns[1:3,]].replace('[\$,]', '', regex=True).astype(float)

In [981]:
main_df2

,Price / sq ft,Sold Price,Sold Price Pre-Pandemic,zipcode
14,955.0,620000.0,0.0,95110
18,963.0,835000.0,0.0,94085
19,924.0,801000.0,0.0,94085
20,1024.0,930000.0,0.0,95054
21,910.0,787000.0,0.0,95054
22,1020.0,882000.0,0.0,95054
23,936.0,850000.0,0.0,95054
24,894.0,805000.0,0.0,95054
25,925.0,840000.0,0.0,95054
31,998.0,650000.0,0.0,95051


In [982]:
main_df2.drop('Sold Price Pre-Pandemic', inplace=True, axis=1)

In [983]:
main_df2

,Price / sq ft,Sold Price,zipcode
14,955.0,620000.0,95110
18,963.0,835000.0,94085
19,924.0,801000.0,94085
20,1024.0,930000.0,95054
21,910.0,787000.0,95054
22,1020.0,882000.0,95054
23,936.0,850000.0,95054
24,894.0,805000.0,95054
25,925.0,840000.0,95054
31,998.0,650000.0,95051


In [984]:
main_df2.info()
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 14 to 55
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Price / sq ft  12 non-null     float64
 1   Sold Price     12 non-null     float64
 2   zipcode        12 non-null     object 
dtypes: float64(2), object(1)
memory usage: 384.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   publishedDate     30 non-null     object 
 1   price             30 non-null     int64  
 2   squareFootage     30 non-null     int64  
 3   sq_Footage_price  30 non-null     float64
 4   zipcode           30 non-null     object 
dtypes: float64(1), int64(2), object(2)
memory usage: 2.5+ KB


In [991]:
# carve out this section to prepare a dataset for the final section 

final_2 = main_df2[['zipcode','Price / sq ft']]
final_2.columns = ['zipcode', 'sq_Footage_price']
final_df = pd.concat([main_df[['zipcode','sq_Footage_price']], final_2], ignore_index=True)
final_df



,zipcode,sq_Footage_price
0,95054,688.0
1,95054,612.0
2,95054,673.0
3,95054,734.0
4,95054,611.0
5,95110,640.0
6,95110,642.0
7,95110,636.0
8,95110,915.0
9,95110,955.0


In [725]:
# backing up a copy of geoz_group2
# geoz_group2_bkp = geoz_group2.copy()
# geoz_group2_bkp

In [739]:
geoz_group2 = main_df2.groupby('zipcode').agg({'Sold Price': np.median, 'Price / sq ft': np.median})

In [740]:
geoz_group2
# geoz_group

,Sold Price,Price / sq ft
zipcode,,
94085,818000.0,943.5
94086,649000.0,1074.0
94087,695000.0,921.0
95051,650000.0,998.0
95054,845000.0,930.5
95110,620000.0,955.0


In [728]:
merged = pd.merge(geoz_group, geoz_group2, on = 'zipcode', how = 'inner')

In [729]:
merged['price_change'] = (merged.price - merged['Sold Price'])/merged['Sold Price']*100

In [730]:
merged['sq_footage_price_change'] = (merged.sq_Footage_price - merged['Price / sq ft'])/merged['Price / sq ft']*100

In [731]:
merged = merged.reset_index()


In [732]:
merged

,zipcode,price,sq_Footage_price,Sold Price,Price / sq ft,price_change,sq_footage_price_change
0,94085,710828.428571,770.428571,818000.0,943.5,-13.101659,-18.343554
1,94086,932629.333333,966.000000,649000.0,1074.0,43.702517,-10.055866
2,94087,591777.600000,816.800000,695000.0,921.0,-14.852144,-11.313789
3,95051,545990.000000,824.200000,650000.0,998.0,-16.001538,-17.414830
4,95054,548932.800000,663.600000,849000.0,951.5,-35.343604,-30.257488
5,95110,609200.000000,757.600000,620000.0,955.0,-1.741935,-20.670157


In [733]:
# first try with 95054 only
# create_map(merged, 'zipcode', mapped_feature = 'price_change', add_text = 'of 1bd/1ba-condos', delimiter = 'zipcode')


In [735]:
# 6 zipcodes and price_change... make commentaries on the lone increase (bucking of trend) in 94086
create_map(merged, 'zipcode', mapped_feature = 'price_change', add_text = 'of 1bd/1ba-condos', delimiter = 'zipcode')


In [736]:
# 6 zipcodes and Sq. footage price_change...
create_map2(merged, 'zipcode', mapped_feature = 'sq_footage_price_change', add_text = 'of 1bd/1ba-condos', delimiter = 'zipcode')


#### Rent Estimate


In [768]:
# 95054
url = "https://realtymole-rental-estimate-v1.p.rapidapi.com/rentalPrice"

querystring = {"bedrooms":"1","squareFootage":"870","compCount":"10","bathrooms":"1","address":"1883 Agnew Rd Unit 318, Santa Clara, CA"}

headers = {
    'x-rapidapi-key': "ca0f4a0c63msheb56ce445093e55p1d9232jsn4658d02eedac",
    'x-rapidapi-host': "realtymole-rental-estimate-v1.p.rapidapi.com"
    }

resp_rent1 = requests.request("GET", url, headers=headers, params=querystring)

print(resp_rent1.text)
rent_estimate_pdf_1 = json_normalize(resp_rent1.json())
rent_estimate_pdf_1.head()

{"rent":1757.48,"rentRangeLow":1299.96,"rentRangeHigh":2215,"longitude":-121.9582871,"latitude":37.3958195,"listings":[{"id":"2200-Agnew-Rd,-Santa-Clara,-CA-95054","formattedAddress":"2200 Agnew Rd, Santa Clara, CA 95054","longitude":-121.963629,"latitude":37.39238,"city":"Santa Clara","state":"CA","zipcode":"95054","price":880,"publishedDate":"2020-12-21T02:47:13.604Z","distance":0.6081091368618204,"daysOld":148.1,"correlation":0.9264,"address":"2200 Agnew Rd","county":"Santa Clara County","bedrooms":1,"bathrooms":1.5,"propertyType":"Condo","squareFootage":985},{"id":"4224-Cheeney-St,-Apt-3,-Santa-Clara,-CA-95054","formattedAddress":"4224 Cheeney St, Apt 3, Santa Clara, CA 95054","longitude":-121.961059,"latitude":37.392734,"city":"Santa Clara","state":"CA","zipcode":"95054","price":1750,"publishedDate":"2021-02-27T03:06:40.853Z","distance":0.4219853793205227,"daysOld":80.09,"correlation":0.8747,"address":"4224 Cheeney St, Apt 3","county":"Santa Clara County","bedrooms":1,"bathrooms":

,rent,rentRangeLow,rentRangeHigh,longitude,latitude,listings
0,1757.48,1299.96,2215,-121.958287,37.39582,"[{'id': '2200-Agnew-Rd,-Santa-Clara,-CA-95054', 'formattedAddress': '2200 Agnew Rd, Santa Clara, CA 95054', 'longitude': -121.963629, 'latitude': 37.39238, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 880, 'publishedDate': '2020-12-21T02:47:13.604Z', 'distance': 0.6081091368618204, 'daysOld': 148.1, 'correlation': 0.9264, 'address': '2200 Agnew Rd', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1.5, 'propertyType': 'Condo', 'squareFootage': 985}, {'id': '4224-Cheeney-St,-Apt-3,-Santa-Clara,-CA-95054', 'formattedAddress': '4224 Cheeney St, Apt 3, Santa Clara, CA 95054', 'longitude': -121.961059, 'latitude': 37.392734, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 1750, 'publishedDate': '2021-02-27T03:06:40.853Z', 'distance': 0.4219853793205227, 'daysOld': 80.09, 'correlation': 0.8747, 'address': '4224 Cheeney St, Apt 3', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 500}, {'id': '2011-Beech-St,-Santa-Clara,-CA-95054', 'formattedAddress': '2011 Beech St, Santa Clara, CA 95054', 'longitude': -121.959699, 'latitude': 37.391703, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 1850, 'publishedDate': '2019-05-02T06:30:03.616Z', 'distance': 0.47495257958309767, 'daysOld': 746.95, 'correlation': 0.8708, 'address': '2011 Beech St', 'county': 'Santa Clara', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Apartment', 'squareFootage': 500}, {'id': '4224-Cheeney-St,-Santa-Clara,-CA-95054', 'formattedAddress': '4224 Cheeney St, Santa Clara, CA 95054', 'longitude': -121.961046, 'latitude': 37.392729, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 1750, 'publishedDate': '2021-02-28T02:56:25.018Z', 'distance': 0.4217723867873227, 'daysOld': 79.1, 'correlation': 0.8545, 'address': '4224 Cheeney St', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Apartment'}, {'id': '1880-Chestnut-St,-Apt-4,-Santa-Clara,-CA-95054', 'formattedAddress': '1880 Chestnut St, Apt 4, Santa Clara, CA 95054', 'longitude': -121.958798, 'latitude': 37.390294, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 2450, 'publishedDate': '2019-04-22T06:55:53.893Z', 'distance': 0.6167496849645617, 'daysOld': 756.93, 'correlation': 0.8517, 'address': '1880 Chestnut St, Apt 4', 'county': 'Santa Clara', 'bedrooms': 2, 'bathrooms': 1, 'propertyType': 'Apartment', 'squareFootage': 900}]"


In [743]:
# 95110
url = "https://realtymole-rental-estimate-v1.p.rapidapi.com/rentalPrice"

querystring = {"bedrooms":"1","compCount":"10","bathrooms":"1","address":"175 W St James St, San Jose, CA 95110"}

headers = {
    'x-rapidapi-key': "ca0f4a0c63msheb56ce445093e55p1d9232jsn4658d02eedac",
    'x-rapidapi-host': "realtymole-rental-estimate-v1.p.rapidapi.com"
    }

resp_rent2 = requests.request("GET", url, headers=headers, params=querystring)

print(resp_rent2.text)
rent_estimate_pdf_2 = json_normalize(resp_rent2.json())
rent_estimate_pdf_2.head()

{"rent":2229.03,"rentRangeLow":2020.74,"rentRangeHigh":2437.32,"longitude":-121.895531,"latitude":37.3379369,"listings":[{"id":"175-W-Saint-James-St,-Unit-702,-San-Jose,-CA-95110","formattedAddress":"175 W Saint James St, Unit 702, San Jose, CA 95110","longitude":-121.895497,"latitude":37.337932,"city":"San Jose","state":"CA","zipcode":"95110","price":1995,"publishedDate":"2020-11-11T02:55:04.424Z","distance":0.0030582579442132176,"daysOld":188.07,"correlation":0.9989,"address":"175 W Saint James St, Unit 702","county":"Santa Clara County","bedrooms":1,"bathrooms":1,"propertyType":"Condo","squareFootage":673},{"id":"175-W-Saint-James-St,-Unit-1402,-San-Jose,-CA-95110","formattedAddress":"175 W Saint James St, Unit 1402, San Jose, CA 95110","longitude":-121.89551,"latitude":37.337937,"city":"San Jose","state":"CA","zipcode":"95110","price":2600,"publishedDate":"2020-07-21T18:51:38.037Z","distance":0.001858669923102646,"daysOld":300.41,"correlation":0.9983,"address":"175 W Saint James St

,rent,rentRangeLow,rentRangeHigh,longitude,latitude,listings
0,2229.03,2020.74,2437.32,-121.895531,37.337937,"[{'id': '175-W-Saint-James-St,-Unit-702,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 702, San Jose, CA 95110', 'longitude': -121.895497, 'latitude': 37.337932, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 1995, 'publishedDate': '2020-11-11T02:55:04.424Z', 'distance': 0.0030582579442132176, 'daysOld': 188.07, 'correlation': 0.9989, 'address': '175 W Saint James St, Unit 702', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 673}, {'id': '175-W-Saint-James-St,-Unit-1402,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 1402, San Jose, CA 95110', 'longitude': -121.89551, 'latitude': 37.337937, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 2600, 'publishedDate': '2020-07-21T18:51:38.037Z', 'distance': 0.001858669923102646, 'daysOld': 300.41, 'correlation': 0.9983, 'address': '175 W Saint James St, Unit 1402', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 796}, {'id': '175-W-Saint-James-St,-Unit-1202,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 1202, San Jose, CA 95110', 'longitude': -121.89551, 'latitude': 37.337937, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 2350, 'publishedDate': '2020-07-21T18:51:16.405Z', 'distance': 0.001858669923102646, 'daysOld': 300.41, 'correlation': 0.9983, 'address': '175 W Saint James St, Unit 1202', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 675}, {'id': '175-W-St-James-St,-Unit-302,-San-Jose,-CA-95110', 'formattedAddress': '175 W St James St, Unit 302, San Jose, CA 95110', 'longitude': -121.89551, 'latitude': 37.337937, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 2200, 'publishedDate': '2020-03-13T19:02:22.571Z', 'distance': 0.001858669923102646, 'daysOld': 430.4, 'correlation': 0.9976, 'address': '175 W St James St, Unit 302', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 673}, {'id': '175-W-James,-San-Jose,-CA-95110', 'formattedAddress': '175 W James, San Jose, CA 95110', 'longitude': -121.895621, 'latitude': 37.338237, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 2000, 'publishedDate': '2020-07-10T06:17:47.767Z', 'distance': 0.034343319685457535, 'daysOld': 311.93, 'correlation': 0.9972, 'address': '175 W James', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 673}]"


In [744]:
# 94085
url = "https://realtymole-rental-estimate-v1.p.rapidapi.com/rentalPrice"

querystring = {"bedrooms":"1","compCount":"10","bathrooms":"1","address":"612 Arcadia Ter #102 SUNNYVALE, CA 94085"}

headers = {
    'x-rapidapi-key': "ca0f4a0c63msheb56ce445093e55p1d9232jsn4658d02eedac",
    'x-rapidapi-host': "realtymole-rental-estimate-v1.p.rapidapi.com"
    }

resp_rent3 = requests.request("GET", url, headers=headers, params=querystring)

print(resp_rent3.text)
rent_estimate_pdf_3 = json_normalize(resp_rent3.json())
rent_estimate_pdf_3.head()

{"rent":2216.78,"rentRangeLow":1916.47,"rentRangeHigh":2517.09,"longitude":-122.0117097,"latitude":37.3929436,"listings":[{"id":"611-Arcadia-Ter,-Unit-102,-Sunnyvale,-CA-94085","formattedAddress":"611 Arcadia Ter, Unit 102, Sunnyvale, CA 94085","longitude":-122.012286,"latitude":37.392852,"city":"Sunnyvale","state":"CA","zipcode":"94085","price":1900,"publishedDate":"2021-02-27T02:55:35.065Z","distance":0.05197892648758487,"daysOld":80.07,"correlation":0.9979,"address":"611 Arcadia Ter, Unit 102","county":"Santa Clara County","bedrooms":1,"bathrooms":1,"propertyType":"Condo","squareFootage":867},{"id":"602-Arcadia-Ter,-Unit-104,-Sunnyvale,-CA-94085","formattedAddress":"602 Arcadia Ter, Unit 104, Sunnyvale, CA 94085","longitude":-122.012758,"latitude":37.393486,"city":"Sunnyvale","state":"CA","zipcode":"94085","price":2345,"publishedDate":"2020-10-27T02:59:54.103Z","distance":0.1106407371958258,"daysOld":203.07,"correlation":0.9953,"address":"602 Arcadia Ter, Unit 104","county":"Santa C

,rent,rentRangeLow,rentRangeHigh,longitude,latitude,listings
0,2216.78,1916.47,2517.09,-122.01171,37.392944,"[{'id': '611-Arcadia-Ter,-Unit-102,-Sunnyvale,-CA-94085', 'formattedAddress': '611 Arcadia Ter, Unit 102, Sunnyvale, CA 94085', 'longitude': -122.012286, 'latitude': 37.392852, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94085', 'price': 1900, 'publishedDate': '2021-02-27T02:55:35.065Z', 'distance': 0.05197892648758487, 'daysOld': 80.07, 'correlation': 0.9979, 'address': '611 Arcadia Ter, Unit 102', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 867}, {'id': '602-Arcadia-Ter,-Unit-104,-Sunnyvale,-CA-94085', 'formattedAddress': '602 Arcadia Ter, Unit 104, Sunnyvale, CA 94085', 'longitude': -122.012758, 'latitude': 37.393486, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94085', 'price': 2345, 'publishedDate': '2020-10-27T02:59:54.103Z', 'distance': 0.1106407371958258, 'daysOld': 203.07, 'correlation': 0.9953, 'address': '602 Arcadia Ter, Unit 104', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 867}, {'id': '608-Arcadia-Ter,-Sunnyvale,-CA-94085', 'formattedAddress': '608 Arcadia Ter, Sunnyvale, CA 94085', 'longitude': -122.012025, 'latitude': 37.39357, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94085', 'price': 2695, 'publishedDate': '2019-11-19T01:41:09.300Z', 'distance': 0.07509922282364584, 'daysOld': 546.13, 'correlation': 0.9946, 'address': '608 Arcadia Ter', 'county': 'Santa Clara', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Apartment', 'squareFootage': 867}, {'id': '642-Blythe-Ct,-Sunnyvale,-CA-94085', 'formattedAddress': '642 Blythe Ct, Sunnyvale, CA 94085', 'longitude': -122.013013, 'latitude': 37.391817, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94085', 'price': 1795, 'publishedDate': '2020-12-20T02:34:32.550Z', 'distance': 0.17033664193793513, 'daysOld': 149.09, 'correlation': 0.9936, 'address': '642 Blythe Ct', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Apartment'}, {'id': '705-San-Conrado-Ter,-Unit-2,-Sunnyvale,-CA-94085', 'formattedAddress': '705 San Conrado Ter, Unit 2, Sunnyvale, CA 94085', 'longitude': -122.014815, 'latitude': 37.392447, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94085', 'price': 2350, 'publishedDate': '2021-03-02T02:37:16.953Z', 'distance': 0.28014714698197585, 'daysOld': 77.09, 'correlation': 0.9904, 'address': '705 San Conrado Ter, Unit 2', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 780}]"


In [745]:
# 95051 
# has one record of 95050 (pandemic period)

url = "https://realtymole-rental-estimate-v1.p.rapidapi.com/rentalPrice"

querystring = {"bedrooms":"1","compCount":"10","bathrooms":"1","address":"150 Saratoga Ave #318 SANTA CLARA, CA 95051"}

headers = {
    'x-rapidapi-key': "ca0f4a0c63msheb56ce445093e55p1d9232jsn4658d02eedac",
    'x-rapidapi-host': "realtymole-rental-estimate-v1.p.rapidapi.com"
    }

resp_rent4 = requests.request("GET", url, headers=headers, params=querystring)

print(resp_rent4.text)
rent_estimate_pdf_4 = json_normalize(resp_rent4.json())
rent_estimate_pdf_4.head()


{"rent":1541.04,"rentRangeLow":1022.01,"rentRangeHigh":2060.07,"longitude":-121.9692176,"latitude":37.3265099,"listings":[{"id":"151-Buckingham-Dr,-Santa-Clara,-CA-95051","formattedAddress":"151 Buckingham Dr, Santa Clara, CA 95051","longitude":-121.97089,"latitude":37.326825,"city":"Santa Clara","state":"CA","zipcode":"95051","price":900,"publishedDate":"2020-09-01T19:13:39.364Z","distance":0.1521392470532965,"daysOld":258.4,"correlation":0.9936,"address":"151 Buckingham Dr","county":"Santa Clara County","bedrooms":1,"bathrooms":1,"propertyType":"Condo","squareFootage":982},{"id":"121-Buckingham-Dr,-Unit-44,-Santa-Clara,-CA-95051","formattedAddress":"121 Buckingham Dr, Unit 44, Santa Clara, CA 95051","longitude":-121.970787,"latitude":37.325978,"city":"Santa Clara","state":"CA","zipcode":"95051","price":2295,"publishedDate":"2019-05-01T00:53:01.783Z","distance":0.15101551285448875,"daysOld":748.17,"correlation":0.991,"address":"121 Buckingham Dr, Unit 44","county":"Santa Clara","bedro

,rent,rentRangeLow,rentRangeHigh,longitude,latitude,listings
0,1541.04,1022.01,2060.07,-121.969218,37.32651,"[{'id': '151-Buckingham-Dr,-Santa-Clara,-CA-95051', 'formattedAddress': '151 Buckingham Dr, Santa Clara, CA 95051', 'longitude': -121.97089, 'latitude': 37.326825, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95051', 'price': 900, 'publishedDate': '2020-09-01T19:13:39.364Z', 'distance': 0.1521392470532965, 'daysOld': 258.4, 'correlation': 0.9936, 'address': '151 Buckingham Dr', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 982}, {'id': '121-Buckingham-Dr,-Unit-44,-Santa-Clara,-CA-95051', 'formattedAddress': '121 Buckingham Dr, Unit 44, Santa Clara, CA 95051', 'longitude': -121.970787, 'latitude': 37.325978, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95051', 'price': 2295, 'publishedDate': '2019-05-01T00:53:01.783Z', 'distance': 0.15101551285448875, 'daysOld': 748.17, 'correlation': 0.991, 'address': '121 Buckingham Dr, Unit 44', 'county': 'Santa Clara', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Single Family', 'squareFootage': 656}, {'id': '151-Buckingham-Dr,-Unit-260,-Santa-Clara,-CA-95051', 'formattedAddress': '151 Buckingham Dr, Unit 260, Santa Clara, CA 95051', 'longitude': -121.971431, 'latitude': 37.327042, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95051', 'price': 1995, 'publishedDate': '2019-07-09T04:55:53.283Z', 'distance': 0.2046872017916427, 'daysOld': 679, 'correlation': 0.9896, 'address': '151 Buckingham Dr, Unit 260', 'county': 'Santa Clara', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Single Family', 'squareFootage': 690}, {'id': '2763-Mauricia-Ave,-Santa-Clara,-CA-95051', 'formattedAddress': '2763 Mauricia Ave, Santa Clara, CA 95051', 'longitude': -121.973468, 'latitude': 37.326957, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95051', 'price': 890, 'publishedDate': '2020-08-25T05:17:13.957Z', 'distance': 0.3795217781955715, 'daysOld': 265.98, 'correlation': 0.9862, 'address': '2763 Mauricia Ave', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 1090}, {'id': '2392-Sutter-Ave,-Santa-Clara,-CA-95050', 'formattedAddress': '2392 Sutter Ave, Santa Clara, CA 95050', 'longitude': -121.964324, 'latitude': 37.328807, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95050', 'price': 1625, 'publishedDate': '2021-05-16T00:50:12.928Z', 'distance': 0.5030190695009451, 'daysOld': 2.17, 'correlation': 0.9836, 'address': '2392 Sutter Ave', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Apartment'}]"


In [746]:
#94086 

url = "https://realtymole-rental-estimate-v1.p.rapidapi.com/rentalPrice"

querystring = {"bedrooms":"1","compCount":"10","bathrooms":"1","address":"420 E Evelyn Ave Unit Z102 SUNNYVALE, CA 94086"}

headers = {
    'x-rapidapi-key': "ca0f4a0c63msheb56ce445093e55p1d9232jsn4658d02eedac",
    'x-rapidapi-host': "realtymole-rental-estimate-v1.p.rapidapi.com"
    }

resp_rent5 = requests.request("GET", url, headers=headers, params=querystring)

print(resp_rent5.text)
rent_estimate_pdf_5 = json_normalize(resp_rent5.json())
rent_estimate_pdf_5.head()

{"rent":2190.3,"rentRangeLow":1696.08,"rentRangeHigh":2684.52,"longitude":-122.0252971,"latitude":37.3756436,"listings":[{"id":"355-E-Evelyn-Ave,-Sunnyvale,-CA-94086","formattedAddress":"355 E Evelyn Ave, Sunnyvale, CA 94086","longitude":-122.02662,"latitude":37.376927,"city":"Sunnyvale","state":"CA","zipcode":"94086","price":3270,"publishedDate":"2020-03-03T06:49:34.670Z","distance":0.1846776173825685,"daysOld":440.92,"correlation":0.9916,"address":"355 E Evelyn Ave","county":"Santa Clara County","bedrooms":1,"bathrooms":1,"propertyType":"Apartment","squareFootage":760},{"id":"284-S-Bayview-Ave,-Sunnyvale,-CA-94086","formattedAddress":"284 S Bayview Ave, Sunnyvale, CA 94086","longitude":-122.026726,"latitude":37.37365,"city":"Sunnyvale","state":"CA","zipcode":"94086","price":1995,"publishedDate":"2021-02-27T03:05:11.173Z","distance":0.2553998182151237,"daysOld":80.08,"correlation":0.9912,"address":"284 S Bayview Ave","county":"Santa Clara County","bedrooms":1,"bathrooms":1,"propertyTy

,rent,rentRangeLow,rentRangeHigh,longitude,latitude,listings
0,2190.3,1696.08,2684.52,-122.025297,37.375644,"[{'id': '355-E-Evelyn-Ave,-Sunnyvale,-CA-94086', 'formattedAddress': '355 E Evelyn Ave, Sunnyvale, CA 94086', 'longitude': -122.02662, 'latitude': 37.376927, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94086', 'price': 3270, 'publishedDate': '2020-03-03T06:49:34.670Z', 'distance': 0.1846776173825685, 'daysOld': 440.92, 'correlation': 0.9916, 'address': '355 E Evelyn Ave', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Apartment', 'squareFootage': 760}, {'id': '284-S-Bayview-Ave,-Sunnyvale,-CA-94086', 'formattedAddress': '284 S Bayview Ave, Sunnyvale, CA 94086', 'longitude': -122.026726, 'latitude': 37.37365, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94086', 'price': 1995, 'publishedDate': '2021-02-27T03:05:11.173Z', 'distance': 0.2553998182151237, 'daysOld': 80.08, 'correlation': 0.9912, 'address': '284 S Bayview Ave', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Single Family', 'squareFootage': 650}, {'id': '464-Bryan-Ave-Apt-F,-Sunnyvale,-CA-94086', 'formattedAddress': '464 Bryan Ave Apt F, Sunnyvale, CA 94086', 'longitude': -122.025144, 'latitude': 37.373201, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94086', 'price': 1895, 'publishedDate': '2020-11-06T03:10:30.805Z', 'distance': 0.2722445105559309, 'daysOld': 193.07, 'correlation': 0.9901, 'address': '464 Bryan Ave Apt F', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 600}, {'id': '464-Bryan-Ave-Apt-D,-Sunnyvale,-CA-94086', 'formattedAddress': '464 Bryan Ave Apt D, Sunnyvale, CA 94086', 'longitude': -122.025144, 'latitude': 37.373201, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94086', 'price': 1895, 'publishedDate': '2020-11-06T03:10:30.804Z', 'distance': 0.2722445105559309, 'daysOld': 193.07, 'correlation': 0.9901, 'address': '464 Bryan Ave Apt D', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 600}, {'id': '464-Bryan-Ave-Apt-E,-Sunnyvale,-CA-94086', 'formattedAddress': '464 Bryan Ave Apt E, Sunnyvale, CA 94086', 'longitude': -122.025144, 'latitude': 37.373201, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94086', 'price': 1895, 'publishedDate': '2020-10-11T02:55:11.129Z', 'distance': 0.2722445105559309, 'daysOld': 219.08, 'correlation': 0.9899, 'address': '464 Bryan Ave Apt E', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 600}]"


In [839]:
#94087 

url = "https://realtymole-rental-estimate-v1.p.rapidapi.com/rentalPrice"

querystring = {"bedrooms":"1","compCount":"10","bathrooms":"1","address":"880 E Fremont Ave #304, SUNNYVALE, CA 94087"}

headers = {
    'x-rapidapi-key': "ca0f4a0c63msheb56ce445093e55p1d9232jsn4658d02eedac",
    'x-rapidapi-host': "realtymole-rental-estimate-v1.p.rapidapi.com"
    }

resp_rent6 = requests.request("GET", url, headers=headers, params=querystring)

print(resp_rent6.text)
rent_estimate_pdf_6 = json_normalize(resp_rent6.json())
rent_estimate_pdf_6.head()

{"rent":2068.81,"rentRangeLow":1803.98,"rentRangeHigh":2333.64,"longitude":-122.0154158,"latitude":37.351578,"listings":[{"id":"880-E-Fremont-Ave,-Apt-110,-Sunnyvale,-CA-94087","formattedAddress":"880 E Fremont Ave, Apt 110, Sunnyvale, CA 94087","longitude":-122.015222,"latitude":37.35183,"city":"Sunnyvale","state":"CA","zipcode":"94087","price":1950,"publishedDate":"2021-02-26T03:10:10.551Z","distance":0.03287912333095577,"daysOld":81.12,"correlation":0.9985,"address":"880 E Fremont Ave, Apt 110","county":"Santa Clara County","bedrooms":1,"bathrooms":1,"propertyType":"Condo","squareFootage":713},{"id":"880-E-Fremont-Ave,-Apt-325,-Sunnyvale,-CA-94087","formattedAddress":"880 E Fremont Ave, Apt 325, Sunnyvale, CA 94087","longitude":-122.015564,"latitude":37.351349,"city":"Sunnyvale","state":"CA","zipcode":"94087","price":1925,"publishedDate":"2020-10-21T03:02:12.666Z","distance":0.02866753832856743,"daysOld":209.13,"correlation":0.9979,"address":"880 E Fremont Ave, Apt 325","county":"Sa

,rent,rentRangeLow,rentRangeHigh,longitude,latitude,listings
0,2068.81,1803.98,2333.64,-122.015416,37.351578,"[{'id': '880-E-Fremont-Ave,-Apt-110,-Sunnyvale,-CA-94087', 'formattedAddress': '880 E Fremont Ave, Apt 110, Sunnyvale, CA 94087', 'longitude': -122.015222, 'latitude': 37.35183, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94087', 'price': 1950, 'publishedDate': '2021-02-26T03:10:10.551Z', 'distance': 0.03287912333095577, 'daysOld': 81.12, 'correlation': 0.9985, 'address': '880 E Fremont Ave, Apt 110', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 713}, {'id': '880-E-Fremont-Ave,-Apt-325,-Sunnyvale,-CA-94087', 'formattedAddress': '880 E Fremont Ave, Apt 325, Sunnyvale, CA 94087', 'longitude': -122.015564, 'latitude': 37.351349, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94087', 'price': 1925, 'publishedDate': '2020-10-21T03:02:12.666Z', 'distance': 0.02866753832856743, 'daysOld': 209.13, 'correlation': 0.9979, 'address': '880 E Fremont Ave, Apt 325', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 713}, {'id': '880-E-Fremont-Ave,-Apt-520,-Sunnyvale,-CA-94087', 'formattedAddress': '880 E Fremont Ave, Apt 520, Sunnyvale, CA 94087', 'longitude': -122.016338, 'latitude': 37.351518, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94087', 'price': 2290, 'publishedDate': '2021-02-27T03:06:40.835Z', 'distance': 0.08187877901961653, 'daysOld': 80.12, 'correlation': 0.9969, 'address': '880 E Fremont Ave, Apt 520', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 733}, {'id': '880-E-Fremont-Ave,-Apt-724,-Sunnyvale,-CA-94087', 'formattedAddress': '880 E Fremont Ave, Apt 724, Sunnyvale, CA 94087', 'longitude': -122.015929, 'latitude': 37.350821, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94087', 'price': 1680, 'publishedDate': '2021-03-21T01:43:41.199Z', 'distance': 0.09572640654605745, 'daysOld': 58.18, 'correlation': 0.9966, 'address': '880 E Fremont Ave, Apt 724', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo'}, {'id': '880-E-Fremont-Ave,-Apt-323,-Sunnyvale,-CA-94087', 'formattedAddress': '880 E Fremont Ave, Apt 323, Sunnyvale, CA 94087', 'longitude': -122.015425, 'latitude': 37.351275, 'city': 'Sunnyvale', 'state': 'CA', 'zipcode': '94087', 'price': 2500, 'publishedDate': '2019-07-09T05:04:01.810Z', 'distance': 0.03373943352362619, 'daysOld': 679.04, 'correlation': 0.9952, 'address': '880 E Fremont Ave, Apt 323', 'county': 'Santa Clara', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 733}]"


#### Combining 6 dataframes into one single dataframe

In [857]:
rent_estimate_pdfs = pd.concat([rent_estimate_pdf_1, rent_estimate_pdf_2, rent_estimate_pdf_3, rent_estimate_pdf_4, rent_estimate_pdf_5, rent_estimate_pdf_6], ignore_index=True)
rent_estimate_pdfs

,rent,rentRangeLow,rentRangeHigh,longitude,latitude,listings
0,1757.48,1299.96,2215.00,-121.958287,37.395820,"[{'id': '2200-Agnew-Rd,-Santa-Clara,-CA-95054', 'formattedAddress': '2200 Agnew Rd, Santa Clara, CA 95054', 'longitude': -121.963629, 'latitude': 37.39238, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 880, 'publishedDate': '2020-12-21T02:47:13.604Z', 'distance': 0.6081091368618204, 'daysOld': 148.1, 'correlation': 0.9264, 'address': '2200 Agnew Rd', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1.5, 'propertyType': 'Condo', 'squareFootage': 985}, {'id': '4224-Cheeney-St,-Apt-3,-Santa-Clara,-CA-95054', 'formattedAddress': '4224 Cheeney St, Apt 3, Santa Clara, CA 95054', 'longitude': -121.961059, 'latitude': 37.392734, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 1750, 'publishedDate': '2021-02-27T03:06:40.853Z', 'distance': 0.4219853793205227, 'daysOld': 80.09, 'correlation': 0.8747, 'address': '4224 Cheeney St, Apt 3', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 500}, {'id': '2011-Beech-St,-Santa-Clara,-CA-95054', 'formattedAddress': '2011 Beech St, Santa Clara, CA 95054', 'longitude': -121.959699, 'latitude': 37.391703, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 1850, 'publishedDate': '2019-05-02T06:30:03.616Z', 'distance': 0.47495257958309767, 'daysOld': 746.95, 'correlation': 0.8708, 'address': '2011 Beech St', 'county': 'Santa Clara', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Apartment', 'squareFootage': 500}, {'id': '4224-Cheeney-St,-Santa-Clara,-CA-95054', 'formattedAddress': '4224 Cheeney St, Santa Clara, CA 95054', 'longitude': -121.961046, 'latitude': 37.392729, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 1750, 'publishedDate': '2021-02-28T02:56:25.018Z', 'distance': 0.4217723867873227, 'daysOld': 79.1, 'correlation': 0.8545, 'address': '4224 Cheeney St', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Apartment'}, {'id': '1880-Chestnut-St,-Apt-4,-Santa-Clara,-CA-95054', 'formattedAddress': '1880 Chestnut St, Apt 4, Santa Clara, CA 95054', 'longitude': -121.958798, 'latitude': 37.390294, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 2450, 'publishedDate': '2019-04-22T06:55:53.893Z', 'distance': 0.6167496849645617, 'daysOld': 756.93, 'correlation': 0.8517, 'address': '1880 Chestnut St, Apt 4', 'county': 'Santa Clara', 'bedrooms': 2, 'bathrooms': 1, 'propertyType': 'Apartment', 'squareFootage': 900}]"
1,2229.03,2020.74,2437.32,-121.895531,37.337937,"[{'id': '175-W-Saint-James-St,-Unit-702,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 702, San Jose, CA 95110', 'longitude': -121.895497, 'latitude': 37.337932, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 1995, 'publishedDate': '2020-11-11T02:55:04.424Z', 'distance': 0.0030582579442132176, 'daysOld': 188.07, 'correlation': 0.9989, 'address': '175 W Saint James St, Unit 702', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 673}, {'id': '175-W-Saint-James-St,-Unit-1402,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 1402, San Jose, CA 95110', 'longitude': -121.89551, 'latitude': 37.337937, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 2600, 'publishedDate': '2020-07-21T18:51:38.037Z', 'distance': 0.001858669923102646, 'daysOld': 300.41, 'correlation': 0.9983, 'address': '175 W Saint James St, Unit 1402', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 796}, {'id': '175-W-Saint-James-St,-Unit-1202,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 1202, San Jose, CA 95110', 'longitude': -121.89551, 'latitude': 37.337937, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 2350, 'publishedDate': '2020-07-21T18:51:16.405Z', 'd

In [749]:
maya.parse(rent_estimate_pdfs.listings[k][i]['publishedDate']).datetime(to_timezone='America/Los_Angeles', naive=False)

datetime.datetime(2019, 7, 8, 22, 4, 1, 810000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>)

In [752]:
rent_estimate_pdfs.head()

,rent,rentRangeLow,rentRangeHigh,longitude,latitude,listings
0,1757.48,1299.96,2215.00,-121.958287,37.395820,"[{'id': '2200-Agnew-Rd,-Santa-Clara,-CA-95054', 'formattedAddress': '2200 Agnew Rd, Santa Clara, CA 95054', 'longitude': -121.963629, 'latitude': 37.39238, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 880, 'publishedDate': '2020-12-21T02:47:13.604Z', 'distance': 0.6081091368618204, 'daysOld': 148.08, 'correlation': 0.9264, 'address': '2200 Agnew Rd', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1.5, 'propertyType': 'Condo', 'squareFootage': 985}, {'id': '4224-Cheeney-St,-Apt-3,-Santa-Clara,-CA-95054', 'formattedAddress': '4224 Cheeney St, Apt 3, Santa Clara, CA 95054', 'longitude': -121.961059, 'latitude': 37.392734, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 1750, 'publishedDate': '2021-02-27T03:06:40.853Z', 'distance': 0.4219853793205227, 'daysOld': 80.06, 'correlation': 0.8747, 'address': '4224 Cheeney St, Apt 3', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 500}, {'id': '2011-Beech-St,-Santa-Clara,-CA-95054', 'formattedAddress': '2011 Beech St, Santa Clara, CA 95054', 'longitude': -121.959699, 'latitude': 37.391703, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 1850, 'publishedDate': '2019-05-02T06:30:03.616Z', 'distance': 0.47495257958309767, 'daysOld': 746.92, 'correlation': 0.8708, 'address': '2011 Beech St', 'county': 'Santa Clara', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Apartment', 'squareFootage': 500}, {'id': '4224-Cheeney-St,-Santa-Clara,-CA-95054', 'formattedAddress': '4224 Cheeney St, Santa Clara, CA 95054', 'longitude': -121.961046, 'latitude': 37.392729, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 1750, 'publishedDate': '2021-02-28T02:56:25.018Z', 'distance': 0.4217723867873227, 'daysOld': 79.07, 'correlation': 0.8545, 'address': '4224 Cheeney St', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Apartment'}, {'id': '1880-Chestnut-St,-Apt-4,-Santa-Clara,-CA-95054', 'formattedAddress': '1880 Chestnut St, Apt 4, Santa Clara, CA 95054', 'longitude': -121.958798, 'latitude': 37.390294, 'city': 'Santa Clara', 'state': 'CA', 'zipcode': '95054', 'price': 2450, 'publishedDate': '2019-04-22T06:55:53.893Z', 'distance': 0.6167496849645617, 'daysOld': 756.9, 'correlation': 0.8517, 'address': '1880 Chestnut St, Apt 4', 'county': 'Santa Clara', 'bedrooms': 2, 'bathrooms': 1, 'propertyType': 'Apartment', 'squareFootage': 900}]"
1,2229.03,2020.74,2437.32,-121.895531,37.337937,"[{'id': '175-W-Saint-James-St,-Unit-702,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 702, San Jose, CA 95110', 'longitude': -121.895497, 'latitude': 37.337932, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 1995, 'publishedDate': '2020-11-11T02:55:04.424Z', 'distance': 0.0030582579442132176, 'daysOld': 188.07, 'correlation': 0.9989, 'address': '175 W Saint James St, Unit 702', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 673}, {'id': '175-W-Saint-James-St,-Unit-1402,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 1402, San Jose, CA 95110', 'longitude': -121.89551, 'latitude': 37.337937, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 2600, 'publishedDate': '2020-07-21T18:51:38.037Z', 'distance': 0.001858669923102646, 'daysOld': 300.41, 'correlation': 0.9983, 'address': '175 W Saint James St, Unit 1402', 'county': 'Santa Clara County', 'bedrooms': 1, 'bathrooms': 1, 'propertyType': 'Condo', 'squareFootage': 796}, {'id': '175-W-Saint-James-St,-Unit-1202,-San-Jose,-CA-95110', 'formattedAddress': '175 W Saint James St, Unit 1202, San Jose, CA 95110', 'longitude': -121.89551, 'latitude': 37.337937, 'city': 'San Jose', 'state': 'CA', 'zipcode': '95110', 'price': 2350, 'publishedDate': '2020-07-21T18:51:16.405Z', '

In [756]:
# rent_estimate_pdfs.listings[0][3].pop('id', '4224-Cheeney-St,-Apt-3,-Santa-Clara,-CA-95054')

'4224-Cheeney-St,-Santa-Clara,-CA-95054'

In [779]:
# rent_estimate_pdfs.listings[0][3]['squareFootage']
print(round(rent_estimate_pdfs.listings[0][0]['price'] / rent_estimate_pdfs.listings[0][0]['squareFootage']*100,0))

89.0


In [886]:
# For some unknown reason, python would still add the last record into the data, need to manually delete the last incomplete record

columns = ['publishedDate', 'price', 'squareFootage', 'sq_Footage_price', 'zipcode']
main_df = pd.DataFrame(columns=columns)
counter = 0
for k in range(len(rent_estimate_pdfs.listings)):
    for i in range(len(rent_estimate_pdfs.listings[k])):
        publishedDate = maya.parse(rent_estimate_pdfs.listings[k][i]['publishedDate']).datetime(to_timezone='America/Los_Angeles', naive=False)
    #     print(publishedDate.date())
        if publishedDate.date() in DatetimeRange(pandemic_dt1, pandemic_dt2):
                try:
                    print(counter)
                    main_df.loc[counter, 'publishedDate'] = publishedDate.date()
                    print(publishedDate.date())
                    main_df.loc[counter, 'price'] = rent_estimate_pdfs.listings[k][i]['price']
                    print( rent_estimate_pdfs.listings[k][i]['price'])
                    main_df.loc[counter, 'squareFootage'] =  rent_estimate_pdfs.listings[k][i]['squareFootage']
                    print(rent_estimate_pdfs.listings[k][i]['squareFootage'])
                    main_df.loc[counter, 'sq_Footage_price'] =  round(rent_estimate_pdfs.listings[k][i]['price'] / rent_estimate_pdfs.listings[k][i]['squareFootage']*1000,0)
                    print(round(rent_estimate_pdfs.listings[k][i]['price'] / rent_estimate_pdfs.listings[k][i]['squareFootage']*1000,0))
                    main_df.loc[counter, 'zipcode'] =  rent_estimate_pdfs.listings[k][i]['zipcode']
                    print(rent_estimate_pdfs.listings[k][i]['zipcode'])
                    counter+=1
                    
                except: 
                    print('squareFootage is not present')    


0
2020-12-20
880
985
893.0
95054
1
2021-02-26
1750
500
3500.0
95054
2
2021-02-27
1750
squareFootage is not present
2
2020-11-10
1995
673
2964.0
95110
3
2020-07-21
2600
796
3266.0
95110
4
2020-07-21
2350
675
3481.0
95110
5
2020-07-09
2000
673
2972.0
95110
6
2021-02-26
1900
867
2191.0
94085
7
2020-10-26
2345
867
2705.0
94085
8
2020-12-19
1795
squareFootage is not present
8
2021-03-01
2350
780
3013.0
94085
9
2020-09-01
900
982
916.0
95051
10
2020-08-24
890
1090
817.0
95051
11
2021-05-15
1625
squareFootage is not present
11
2021-02-26
1995
650
3069.0
94086
12
2020-11-05
1895
600
3158.0
94086
13
2020-11-05
1895
600
3158.0
94086
14
2020-10-10
1895
600
3158.0
94086
15
2021-02-25
1950
713
2735.0
94087
16
2020-10-20
1925
713
2700.0
94087
17
2021-02-26
2290
733
3124.0
94087
18
2021-03-20
1680
squareFootage is not present


In [887]:
main_df = main_df[:-1]


In [888]:
# rent_estimate_pdfs.listings[5][3]

In [892]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   publishedDate     18 non-null     object 
 1   price             18 non-null     int64  
 2   squareFootage     18 non-null     int64  
 3   sq_Footage_price  18 non-null     float64
 4   zipcode           18 non-null     object 
dtypes: float64(1), int64(2), object(2)
memory usage: 864.0+ bytes


In [891]:
main_df.price = main_df.fillna(0).price.astype(int)
main_df.squareFootage =  main_df.fillna(0).squareFootage.astype(int)
main_df.sq_Footage_price = main_df.fillna(0).sq_Footage_price.astype(float)


In [893]:
main_df

,publishedDate,price,squareFootage,sq_Footage_price,zipcode
0,2020-12-20,880,985,893.0,95054
1,2021-02-26,1750,500,3500.0,95054
2,2020-11-10,1995,673,2964.0,95110
3,2020-07-21,2600,796,3266.0,95110
4,2020-07-21,2350,675,3481.0,95110
5,2020-07-09,2000,673,2972.0,95110
6,2021-02-26,1900,867,2191.0,94085
7,2020-10-26,2345,867,2705.0,94085
8,2021-03-01,2350,780,3013.0,94085
9,2020-09-01,900,982,916.0,95051


In [894]:
geoz_group = main_df.groupby('zipcode').agg({'price': np.median, 'sq_Footage_price': np.median})
geoz_group

,price,sq_Footage_price
zipcode,,
94085,2345,2705.0
94086,1895,3158.0
94087,1950,2735.0
95051,895,866.5
95054,1315,2196.5
95110,2175,3119.0


In [895]:
columns = ['publishedDate', 'price', 'squareFootage', 'sq_Footage_price', 'zipcode']
main_df2 = pd.DataFrame(columns=columns)
counter = 0
for k in range(len(rent_estimate_pdfs.listings)):
    for i in range(len(rent_estimate_pdfs.listings[k])):
        publishedDate = maya.parse(rent_estimate_pdfs.listings[k][i]['publishedDate']).datetime(to_timezone='America/Los_Angeles', naive=False)
    #     print(publishedDate.date())
        if publishedDate.date() not in DatetimeRange(pandemic_dt1, pandemic_dt2):
                try:
                    print(counter)
                    main_df2.loc[counter, 'publishedDate'] = publishedDate.date()
                    print(publishedDate.date())
                    main_df2.loc[counter, 'price'] = rent_estimate_pdfs.listings[k][i]['price']
                    print( rent_estimate_pdfs.listings[k][i]['price'])
                    main_df2.loc[counter, 'squareFootage'] =  rent_estimate_pdfs.listings[k][i]['squareFootage']
                    print(rent_estimate_pdfs.listings[k][i]['squareFootage'])
                    main_df2.loc[counter, 'sq_Footage_price'] =  round(rent_estimate_pdfs.listings[k][i]['price'] / rent_estimate_pdfs.listings[k][i]['squareFootage']*1000,0)
                    print(round(rent_estimate_pdfs.listings[k][i]['price'] / rent_estimate_pdfs.listings[k][i]['squareFootage']*1000,0))
                    main_df2.loc[counter, 'zipcode'] =  rent_estimate_pdfs.listings[k][i]['zipcode']
                    print(rent_estimate_pdfs.listings[k][i]['zipcode'])
                    counter+=1
                    
                except: 
                    print('squareFootage is not present')    


0
2019-05-01
1850
500
3700.0
95054
1
2019-04-21
2450
900
2722.0
95054
2
2020-03-13
2200
673
3269.0
95110
3
2019-11-18
2695
867
3108.0
94085
4
2019-04-30
2295
656
3498.0
95051
5
2019-07-08
1995
690
2891.0
95051
6
2020-03-02
3270
760
4303.0
94086
7
2019-07-08
2500
733
3411.0
94087


In [896]:
main_df2

,publishedDate,price,squareFootage,sq_Footage_price,zipcode
0,2019-05-01,1850,500,3700.0,95054
1,2019-04-21,2450,900,2722.0,95054
2,2020-03-13,2200,673,3269.0,95110
3,2019-11-18,2695,867,3108.0,94085
4,2019-04-30,2295,656,3498.0,95051
5,2019-07-08,1995,690,2891.0,95051
6,2020-03-02,3270,760,4303.0,94086
7,2019-07-08,2500,733,3411.0,94087


In [898]:
main_df2.price = main_df2.fillna(0).price.astype(int)
main_df2.squareFootage =  main_df2.fillna(0).squareFootage.astype(int)
main_df2.sq_Footage_price = main_df2.fillna(0).sq_Footage_price.astype(float)


In [899]:
geoz_group2 = main_df2.groupby('zipcode').agg({'price': np.median, 'sq_Footage_price': np.median})
geoz_group2

,price,sq_Footage_price
zipcode,,
94085,2695,3108.0
94086,3270,4303.0
94087,2500,3411.0
95051,2145,3194.5
95054,2150,3211.0
95110,2200,3269.0


In [917]:
merged = pd.merge(geoz_group, geoz_group2, on = 'zipcode', how = 'inner', suffixes = ("", "_pre"))
merged['price_change_in_rent'] = (merged.price - merged['price_pre'])/merged['price_pre']*100
merged['price_change_in_rent_per_sqft'] = (merged.sq_Footage_price - merged['sq_Footage_price_pre'])/merged['sq_Footage_price_pre']*100
merged = merged.reset_index()
merged

,zipcode,price,sq_Footage_price,price_pre,sq_Footage_price_pre,price_change_in_rent,price_change_in_rent_per_sqft
0,94085,2345,2705.0,2695,3108.0,-12.987013,-12.966538
1,94086,1895,3158.0,3270,4303.0,-42.048930,-26.609342
2,94087,1950,2735.0,2500,3411.0,-22.000000,-19.818235
3,95051,895,866.5,2145,3194.5,-58.275058,-72.875254
4,95054,1315,2196.5,2150,3211.0,-38.837209,-31.594519
5,95110,2175,3119.0,2200,3269.0,-1.136364,-4.588559


#### customizing create_map for rent estimate price changes

In [907]:
def create_map3(table, zips, mapped_feature, add_text = '', delimiter = ''):
    # reading of the updated GeoJSON file
    sf_geo = r'updated-file.json'
    # initiating a Folium map with 95054's latitude and longitude
    m = folium.Map(location = [37.393, -121.961], zoom_start = 11) 
    # creating a choropleth map
    folium.Choropleth(
        geo_data = sf_geo,
        fill_opacity = 0.54,
        line_opacity = 0.38,
        data = table,
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.ZCTA5CE10',
        # first element contains location information, second element contains feature of interest
        columns = [zips, mapped_feature],
        fill_color = 'PiYG',
        threshold_scale = [-60, -40, -20, 0, 10, 20, 30],
        legend_name = (' ').join(mapped_feature.split('_')).title() + ' ' + add_text + ' across SF Bay Area by ' + delimiter
    ).add_to(m)
#     folium.LayerControl().add_to(m)
    # save map with filename based on the feature of interest
    display(m)
    m.save(outfile = mapped_feature + '_map.html')

In [911]:
def create_map4(table, zips, mapped_feature, add_text = '', delimiter = ''):
    # reading of the updated GeoJSON file
    sf_geo = r'updated-file.json'
    # initiating a Folium map with 95054's latitude and longitude
    m = folium.Map(location = [37.393, -121.961], zoom_start = 11) 
    # creating a choropleth map
    folium.Choropleth(
        geo_data = sf_geo,
        fill_opacity = 0.54,
        line_opacity = 0.38,
        data = table,
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.ZCTA5CE10',
        # first element contains location information, second element contains feature of interest
        columns = [zips, mapped_feature],
        fill_color = 'RdYlGn',
        threshold_scale = [-74, -37, -19, 0, 10, 20, 30],
        legend_name = (' ').join(mapped_feature.split('_')).title() + ' ' + add_text + ' across SF Bay Area by ' + delimiter
    ).add_to(m)
#     folium.LayerControl().add_to(m)
    # save map with filename based on the feature of interest
    display(m)
    m.save(outfile = mapped_feature + '_map.html')

In [920]:
create_map3(merged, 'zipcode', mapped_feature = 'price_change_in_rent', add_text = 'of 1bd/1ba-condos', delimiter = 'zipcode')

In [919]:
create_map4(merged, 'zipcode', mapped_feature = 'price_change_in_rent_per_sqft', add_text = 'of 1bd/1ba-condos', delimiter = 'zip')

#### Last part of the project is to provide a calculator on the percentile of desired selling price in the entire history of real estate transactions for 1-bed/1-ba condos in the county of Santa Clara. The lookup should be also available to find out its percentile given the desired selling sq footage price.

In [1007]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [1048]:
# x = 95
x = 89

In [1049]:
final_df.groupby('zipcode').agg({'sq_Footage_price': percentile(x)})

,sq_Footage_price
zipcode,
94085,928.68
94086,1129.61
94087,921.00
95051,998.00
95054,1011.60
95110,955.00


In [1056]:
santa_clara_county_level = final_df.reset_index().drop('zipcode', axis = 1).agg(percentile(x)).to_frame()[1:]
santa_clara_county_level.columns = ['Santa_Clara_County']
santa_clara_county_level

,Santa_Clara_County
sq_Footage_price,998.0


In [1065]:
desired_selling_price_of_sq_footage = 998.0

In [1066]:
# county level 
percentile = stats.percentileofscore(final_df['sq_Footage_price'], desired_selling_price_of_sq_footage)
print(percentile)

89.28571428571429
